## 폰트 설정

In [ ]:
# 시각화를 위한 폰트설정
import platform
import matplotlib.pyplot as plt

system_name = platform.system()

if system_name == "Darwin" :
    font_family = "AppleGothic"
elif system_name == "Windows":
    font_family = "Malgun Gothic"
else:
    # Linux
    !apt -qq -y install fonts-nanum > /dev/null
    import matplotlib.font_manager as fm

    fontpath = '/usr/share/fonts/truetype/nanum/NanumBarunGothic.ttf'
    font = fm.FontProperties(fname=fontpath, size=9)
    fm._rebuild()
    font_family = "NanumBarunGothic"
    
# 폰트설정
plt.rc("font", family=font_family)
# 마이너스폰트 설정
plt.rc("axes", unicode_minus=False)
# 그래프 스타일 설정
plt.style.use("ggplot")


# 그래프에 retina display 적용
%config InlineBackend.figure_format = 'retina'

In [ ]:
# 데이터 분석을 위해 pandas 불러오기
import pandas as pd

In [ ]:
pd.Series([1, 3, 5, -7, 9]).plot(title="한글폰트 확인")

## 라이브러리 로드

In [ ]:
# 주석을 풀고 설치해 주세요. 주석을 푸는 방법은 아래 코드의 맨 앞에 있는 #을 지워주시면 됩니다.
# !pip install -U finance-datareader

In [ ]:
import FinanceDataReader as fdr
fdr.__version__

## 전체 종목 가져오기
* 종목코드를 구하기 위해 전체 종목 가져오기

In [ ]:
df_krx = fdr.StockListing('KRX')
df_krx.head()

### seaborn 을 통한 빈도수 시각화

In [ ]:
import seaborn as sns

In [ ]:
sns.countplot(data=df_krx, y="Region", hue="Market")

In [ ]:
df_sub_region = df_krx[df_krx["Region"].isin(["서울특별시", "경기도", "인천광역시"])]

In [ ]:
# 수도권에 있는 Region 별 Market 의 빈도수를 시각화 해보세요.


In [ ]:
# 섹터별 지역을 시각화 해보세요

df_krx["Sector"].value_counts().head(20)

In [ ]:
df_sub_sector = df_krx[df_krx["Sector"] == "?"]

In [ ]:
sns.countplot(data=df_sub_sector, y="Region", hue="Market")

## 주가 이름으로 종목 데이터 받아오기

In [ ]:
# 주가 이름으로 종목코드 받아오는 함수 만들기
def item_code_by_item_name(item_name):
    name_list = df_krx.loc[df_krx["Name"] == item_name, "Symbol"].tolist()
    if len(name_list) > 0 :
        item_code = name_list[0]
        return item_code
    else:
        return False

In [ ]:
# 종목명으로 일별 시세 수집
def find_item_list(item_name, year=2020):
    item_code = item_code_by_item_name(item_name)
    df = fdr.DataReader(item_code, str(year))
    return df

## 개별종목 일별시세 가져오기

In [ ]:
카카오 = find_item_list("카카오")
카카오.head(2)

In [ ]:
씨젠 = find_item_list("씨젠")
씨젠.head(2)

In [ ]:
신풍제약 = find_item_list("신풍제약")
신풍제약.head(2)

In [ ]:
현대차 = find_item_list("현대차")
현대차.head(2)

## 판다스 내장 plot 으로 그리기

In [ ]:
ohlc_col = ['Open', 'High', 'Low', 'Close']
ohlc_col

In [ ]:
카카오[ohlc_col].plot(figsize=(15, 4), title="카카오")

In [ ]:
카카오["Volume"].plot(figsize=(15, 4))

In [ ]:
신풍제약[ohlc_col].plot(figsize=(15, 4), title="신풍제약")

In [ ]:
씨젠[ohlc_col].plot(figsize=(15, 4), title="씨젠")

## mplfinace

* https://github.com/matplotlib/mplfinance

In [ ]:
# !pip install --upgrade mplfinance

In [ ]:
import mplfinance as mpf

mpf.__version__

In [ ]:
mpf.plot(카카오[-30:], figsize=(15, 4), volume=True)

In [ ]:
mpf.plot(카카오[-30:], figsize=(15, 4), type='candle', volume=True)

In [ ]:
mpf.plot(카카오[-100:], type='candle', mav=(3,6,9), figsize=(15, 4), volume=True)

## 볼린저 밴드

볼린저 밴드(Bollinger Bands)는 1980년대 존 볼린저가 개발하고 2011년 상표권 취득한 주가 기술적 분석 도구이다. 트레이딩 밴드 개념에서 발전된 볼린저 밴드와 상대지표 그리고 대역폭은 이전 거래와의 상대적 가격의 높음과 낮음을 판단하는 데 사용된다. 볼린저 밴드는 켈트너 채널과 비슷한 변동성 지표이다. 볼린저 밴드는 주가가 이동평균선 중심으로 표준편차 범위 안에서 움직인다는 전제로 개발되었다


볼린저 밴드는 다음과 같이 구성된다.
* N기간 동안의 이동평균(MA)
* 이동평균 위의 K배 표준편차 상위 밴드
* 이동평균 아래의 K배 표준편차 하위 밴드

일반적으로 N과 K의 값은 20과 2이다. 평균값의 기본 선택의 단순 이동 평균이지만, 필요에 따라 다른 종류의 평균값을 쓸 수 있다. 지수 이동 평균(Exponential moving averages)은 일반적인 대안이다. 대개 중간 밴드와 표준 편차의 계산을 위해 같은 기간을 사용한다.



* 출처 : https://ko.wikipedia.org/wiki/%EB%B3%BC%EB%A6%B0%EC%A0%80_%EB%B0%B4%EB%93%9C

In [ ]:
def show_bband(df):
    """
    볼린저 밴드
    N기간 동안의 이동평균(MA)
    이동평균 위의 K배 표준편차 상위 밴드
    이동평균 아래의 K배 표준편차 하위 밴드
    일반적으로 N과 K의 값은 20과 2
    """
    plt.style.use("seaborn-whitegrid")
    df['MA20'] = df['Close'].rolling(window=20).mean()
    df['stddev'] = df['Close'].rolling(window=20).std()
    df['upper'] = df['MA20'] + (df['stddev'] * 2)
    df['lower'] = df['MA20'] - (df['stddev'] * 2)
    df = df[19:].copy()
    
    plt.figure(figsize=(9, 5))
    plt.plot(df.index, df['Close'], color='#0000ff', label='Close')
    plt.plot(df.index, df['upper'], 'r--', label = 'Upper band')
    plt.plot(df.index, df['MA20'], 'k:', label='Moving average 20')
    plt.plot(df.index, df['lower'], 'c--', label = 'Lower band')
    plt.fill_between(df.index, df['upper'], df['lower'], color='0.9')
    plt.legend(loc='best')
    plt.title('Bollinger Band (20 day, 2 std)')
    plt.show()

In [ ]:
show_bband(카카오)